In [16]:
import numpy as np
import tensorflow as tf
import random
import sklearn
from sklearn import metrics
from tensorflow.keras.utils import plot_model
from tensorflow.keras import backend as K

In [14]:
path = './dataset/mnist.npz'
f = np.load(path)
x_train, y_train = f['x_train'], f['y_train']
x_test, y_test = f['x_test'], f['y_test']
f.close()

h = x_train.shape[1] // 2
w = x_train.shape[2] // 2

x_train = np.expand_dims(x_train, axis=-1)
x_train = tf.image.resize(x_train, [h, w]).numpy()  # if we want to resize
x_test = np.expand_dims(x_test, axis=-1)
x_test = tf.image.resize(x_test, [h, w]).numpy()  # if we want to resize

#测试也是255归一化的数据，请不要改归一化
x_train = x_train / 255.
x_test = x_test/ 255.

num_classes = 10

In [ ]:
print(x_train)

In [ ]:
'''
(1) 正例（同一数字对）8000个、反例（不同数字对）9000个。正例中，10个数字类型各占1/10，800个。
(2) 反例中，不同数字对的所有组合共C^2_10=45种，要求比例也为相同，即反例中，45种组合每个组合比例为1/45，200个。
'''
def make_batch(batch_size,dataset):

    return (input_a, input_b), label

In [7]:
#train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train))
#val_set = tf.data.Dataset.from_tensor_slices((x_test, y_test))
train_set = [x_train, y_train]

In [12]:
print(y_train[0])

5


In [20]:
#2. 对数据进行混洗
s_index=np.arange(len(y_train))
np.random.shuffle(s_index)
s_y_train=y_train[s_index]
s_x_train=x_train[s_index]

#3. 将MNIST数据按类型连续排列，np.where返回满足条件的多维数组中的元素的坐标，
#返回坐标的每一维合成一个tuple。标签只有一个维度，固选[0]
index1 = [np.where(s_y_train == i)[0] for i in range(num_classes)]
len_digits=[len(index1[i]) for i in range(num_classes)]

In [ ]:
print(s_y_train[0], s_x_train[0])

In [23]:
def train_data_gen(data, index):
    pairs_l = []
    pairs_r = []
    labels = []

    for c in range(num_classes):
        ran = random.randrange(1, len_digits[c])# for each class: [1, classnum) rand list
        for i in range(800):
            i1, i2 = index[c][i], index[c][(i + ran)%len_digits[c]]
            pairs_l.append(data[i1])
            pairs_r.append(data[i2])
            labels.append(0) # add positive samples (overall 800*10)
    for c in range(num_classes):
        for i in range(c): # change c to num_classes
            if c == i:
                continue

            for _ in range(200):
                ran1 = random.randrange(1, len_digits[c])
                ran2 = random.randrange(1, len_digits[i])
                i1, i2 = index[c][ran1], index[i][ran2]
                pairs_l.append(data[i1])
                pairs_r.append(data[i2])
                labels.append(1) # add negative samples (overall 200*45???)

    return (np.array(pairs_l),np.array(pairs_r)), np.array(labels).astype('float32')

In [24]:
train_pairs, train_y = train_data_gen(s_x_train, index1)
# s_index=np.arange(len(y_test))
s_index=np.arange(len(train_y))
np.random.shuffle(s_index)
train_pairs=(train_pairs[0][s_index],train_pairs[1][s_index])
train_y=train_y[s_index]